In [2]:
import import_ipynb
from preprocessing import *

importing Jupyter notebook from preprocessing.ipynb


In [4]:
def evaluate(model, samples, distance_function=dist, p_dist=2):
    
    total = 0
    correct = 0
    
    for sample in samples:
        q, p, n = sample['query'], sample['pos'], sample['neg']
        q = model(q)
        p = model(p)
        n = model(n)
        q = q[0]
        p = p[0]
        n = n[0]
        if(dist(q, p, p_dist) < dist(q, n, p_dist)):
            correct += 1
        total += 1
    
    print("{} samples in total. Correctly identified: {}, accuracy: {:.3f}".format(total, correct, correct/total))

    

In [5]:
def to_cuda(samples):
    cuda = torch.device('cuda:0')
    for sample in samples:
        q = sample['query']
        p = sample['pos']
        n = sample['neg']
        q = q.to(cuda)
        p = p.to(cuda)    
        n = n.to(cuda) 
        sample['name'] = sample['name']
        sample['query'] = q
        sample['pos'] = p
        sample['neg'] = n
    return samples

In [7]:
def train(model, train_set, test_set, optimizer, dist_function=dist, n_epochs=1, g=None, p_dist=2):
    
    for i in range(n_epochs):
        total_loss = 0
        for sample in train_set:
            q = sample['query']
            p = sample['pos']
            n = sample['neg']
            
            q = model(q)
            p = model(p)
            n = model(n)
            q = q[0]
            p = p[0]
            n = n[0]
            dist1 = dist_function(q, p, p_dist)
            dist2 = dist_function(q, n, p_dist)
            loss = dist1 - dist2 + g
            if(loss > 0):
                loss.backward()
                optimizer.step()
                total_loss += loss
                
        print("Epoch {} completed. Loss: {}".format(i+1, total_loss / len(train_set)))
        evaluate(model, train_set, p_dist)
        evaluate(model, test_set, p_dist)
        

9
